In [2]:
import os
import dotenv
from pathlib import Path

from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.document_loaders.text import TextLoader
from langchain_community.document_loaders import (
    WebBaseLoader, 
    PyPDFLoader, 
    Docx2txtLoader,
)


from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore



from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI

# from langchain_anthropic import ChatAnthropic
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

dotenv.load_dotenv()

USER_AGENT environment variable not set, consider setting it to identify your requests.


True

In [3]:
import os, math
from typing import List, Optional, Dict, Any
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import pinecone_text
from pinecone_text.sparse import BM25Encoder


In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings
from langchain_core.messages import HumanMessage
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Debug: Check if .env file is being loaded
print("Current working directory:", os.getcwd())
print("Files in current directory:", os.listdir('.'))
print("Looking for .env file...")

# Get the API key from environment variables
api_key_google = os.getenv("GEMINI_API_KEY")

# Debug: Check API key
if api_key_google:
    print(f"✅ API key found: {api_key_google[:10]}...{api_key_google[-5:]}")
    print(f"API key length: {len(api_key_google)}")
else:
    print("❌ API key not found!")
    print("Available environment variables:")
    for key in os.environ.keys():
        if 'GOOGLE' in key.upper() or 'API' in key.upper() or 'KEY' in key.upper():
            print(f"  {key}")
    
    # Try alternative names
    alt_keys = ['GOOGLE_GENAI_API_KEY', 'GEMINI_API_KEY', 'GOOGLE_AI_API_KEY']
    for alt_key in alt_keys:
        alt_value = os.getenv(alt_key)
        if alt_value:
            print(f"Found alternative key {alt_key}: {alt_value[:10]}...")
            api_key_google = alt_value
            break

if not api_key_google:
    print("\n❌ No API key found. Please check:")
    print("1. Your .env file exists in the current directory")
    print("2. Your .env file contains: api_key_google=your_actual_key")
    print("3. No spaces around the = sign")
    print("4. No quotes around the key unless they're part of the key")
    exit()

try:
    print("\n🔄 Initializing ChatGoogleGenerativeAI...")
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-pro", 
        google_api_key=api_key_google,
        temperature=0.7
    )
    print("✅ Model initialized successfully!")
    
    print("\n🔄 Sending test message...")
    messages = [HumanMessage(content="Hello, world! Please respond with just 'Hello back!'")]
    result = llm.invoke(messages)
    
    print("✅ Response received:")
    print(f"Type: {type(result)}")
    print(f"Content: {result.content}")
    
except Exception as e:
    print(f"❌ Error occurred: {type(e).__name__}")
    print(f"Error message: {str(e)}")
    
    # Common error troubleshooting
    if "401" in str(e) or "Unauthorized" in str(e):
        print("\n🔍 This looks like an authentication error:")
        print("- Check if your API key is correct")
        print("- Make sure your Google AI API key is active")
        print("- Visit: https://makersuite.google.com/app/apikey")
    
    elif "403" in str(e) or "Forbidden" in str(e):
        print("\n🔍 This looks like a permissions error:")
        print("- Your API key might not have access to Gemini Pro")
        print("- Check your API quotas and billing")
    
    elif "400" in str(e) or "Bad Request" in str(e):
        print("\n🔍 This looks like a bad request:")
        print("- The model name might be incorrect")
        print("- Try 'gemini-1.5-pro' or 'gemini-1.5-flash'")
    
    elif "ImportError" in str(e) or "ModuleNotFoundError" in str(e):
        print("\n🔍 This looks like a missing dependency:")
        print("- Run: pip install langchain-google-genai google-generativeai")
    
    else:
        print("\n🔍 Unexpected error. Full traceback:")
        import traceback
        traceback.print_exc()

Current working directory: /Users/brandon/RAG-Chatbot
Files in current directory: ['chroma_store', 'requirements.txt', 'ubc-pair-grade-data-master', 'RAG-Chatbot.code-workspace', 'ragbot', '.gitignore', '.env', 'rag_langchain.ipynb', 'error.ipynb']
Looking for .env file...
✅ API key found: AIzaSyDcOv...Yug6I
API key length: 39

🔄 Initializing ChatGoogleGenerativeAI...
✅ Model initialized successfully!

🔄 Sending test message...


I0000 00:00:1756585553.988645   81354 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1756585553.994128   81354 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


✅ Response received:
Type: <class 'langchain_core.messages.ai.AIMessage'>
Content: Hello back


In [5]:
from pathlib import Path
from langchain_community.document_loaders import (
    CSVLoader,
    Docx2txtLoader,
    PyPDFLoader,
    TextLoader,
)

doc_paths = ["/Users/brandon/Downloads/ubc-pair-grade-data-master"]
docs = []
files_to_process = []

# First, gather all file paths from the initial `doc_paths`
for path_str in doc_paths:
    p = Path(path_str)
    if not p.exists():
        print(f"Path does not exist: {p}")
        continue
    
    if p.is_dir():
        # If it's a directory, find all files inside it recursively
        files_to_process.extend(p.rglob("*"))
    elif p.is_file():
        # If it's a single file, add it to the list
        files_to_process.append(p)

# Now, process each collected file path
for file_path in files_to_process:
    if not file_path.is_file():
        continue  # Skip any directories that might have been included

    try:
        loader = None
        # Check the file extension and select the appropriate loader
        if file_path.suffix == ".csv":
            loader = CSVLoader(file_path=str(file_path))
        elif file_path.suffix == ".pdf":
            loader = PyPDFLoader(str(file_path))
        elif file_path.suffix == ".docx":
            loader = Docx2txtLoader(str(file_path))
        elif file_path.suffix in [".txt", ".md"]:
            loader = TextLoader(str(file_path))
        else:
            # You can uncomment the line below to see which files are being skipped.
            print(f"Skipping unsupported file type: {file_path.name}")
            continue

        # Load the document and add it to our list
        if loader:
            docs.extend(loader.load())

    except Exception as e:
        print(f"Error loading document {file_path.name}: {e}")

# You can check how many documents were loaded
print(f"Successfully loaded {len(docs)} documents.")


url = "https://docs.streamlit.io/develop/quick-reference/release-notes"
try:
    loader = WebBaseLoader(url)
    docs.extend(loader.load())

except Exception as e:
    print(f"Error loading document from {url}: {e}")


Skipping unsupported file type: .DS_Store
Skipping unsupported file type: UBCV_subjects.json
Skipping unsupported file type: pair-reports-UBC-instructor-corrections.json
Skipping unsupported file type: UBCO_subjects.json
Skipping unsupported file type: .DS_Store
Skipping unsupported file type: .DS_Store
Skipping unsupported file type: .DS_Store
Successfully loaded 482448 documents.


In [6]:
# Split docs

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)

document_chunks = text_splitter.split_documents(docs)

In [7]:
import os, nltk
os.environ["NLTK_DATA"] = os.path.expanduser("~/nltk_data")
nltk.data.path.append(os.path.expanduser("~/nltk_data"))
nltk.download("stopwords")  # should print True

from nltk.corpus import stopwords
from pinecone_text.sparse import BM25Encoder

bm25 = BM25Encoder(stopwords=set(stopwords.words("english")))
bm25.fit([d.page_content for d in document_chunks])
 # Stopwords are the common words in a language that usually carry little meaning by themselves.
# BM25 (and other sparse retrieval methods like TF-IDF) rank documents by token frequency.#
# If you don’t remove stopwords, words like “the” or “is” dominate counts but add no signal.

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/brandon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


TypeError: BM25Encoder.__init__() got an unexpected keyword argument 'stopwords'

In [8]:
# ---- Hybrid indexing (first 10% in order) with batching + retries (DOTPRODUCT index) ----
import os, re, math, time, random
from typing import List, Iterable
from pinecone import Pinecone, ServerlessSpec
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# ======= Config =======
FRACTION = 0.10            # first 10% of chunks; set 1.0 for full
EMBED_INIT_BATCH = 64      # start batch for Gemini
EMBED_MIN_BATCH  = 8       # min batch if we see 504/429/etc.
EMBED_MAX_RETRIES = 6
UPSERT_BATCH = 100
NAMESPACE = "ubc-grades"

# ======= Preconditions =======
assert os.getenv("GEMINI_API_KEY"), "Missing GEMINI_API_KEY"
assert os.getenv("PINECONE_API_KEY"), "Missing PINECONE_API_KEY"
assert "document_chunks" in globals() and len(document_chunks) > 0, "document_chunks empty"

# ---- Dense (Gemini) ----
emb = GoogleGenerativeAIEmbeddings(
    model="models/text-embedding-004",
    google_api_key=os.getenv("GEMINI_API_KEY")
)

def l2_normalize(vec: List[float]) -> List[float]:
    n = math.sqrt(sum(v*v for v in vec)) or 1.0
    return [v / n for v in vec]

def adaptive_embed_documents(
    emb: GoogleGenerativeAIEmbeddings,
    texts: List[str],
    init_batch: int = EMBED_INIT_BATCH,
    min_batch: int = EMBED_MIN_BATCH,
    max_retries: int = EMBED_MAX_RETRIES,
    base_sleep: float = 1.0,
    jitter: float = 0.25,
    progress_every: int = 1000,
    normalize: bool = True,
) -> List[List[float]]:
    out: List[List[float]] = []
    i, B, N = 0, init_batch, len(texts)
    while i < N:
        batch = texts[i : min(i+B, N)]
        attempt = 0
        while True:
            try:
                vecs = emb.embed_documents(batch)
                if normalize:
                    vecs = [l2_normalize(v) for v in vecs]
                out.extend(vecs)
                i += len(batch)
                if i % progress_every == 0 or i == N:
                    print(f"Embedded {i}/{N} (batch={B})")
                if B < init_batch:  # gentle ramp back up
                    B = min(init_batch, B*2)
                break
            except Exception as e:
                err = str(e).lower()
                transient = any(t in err for t in ["504", "deadline", "timeout", "503", "500", "429"])
                attempt += 1
                if transient and B > min_batch:
                    B = max(min_batch, B // 2)
                    print(f"[warn] {e} -> reduce batch to {B} and retry…")
                    time.sleep(base_sleep + random.random()*jitter)
                    continue
                if transient and attempt <= max_retries:
                    sleep = base_sleep*(2**(attempt-1)) + random.random()*jitter
                    print(f"[warn] {e} -> retry {attempt}/{max_retries} in {sleep:.1f}s…")
                    time.sleep(sleep)
                    continue
                raise
    return out

# ---- Take the first 10% in order ----
N_total = len(document_chunks)
k = max(1, int(N_total * FRACTION))
indices = list(range(k))  # first k chunk indices
print(f"Indexing first {k}/{N_total} chunks (fraction={FRACTION:.2f})")

sampled_chunks = [document_chunks[i] for i in indices]
corpus_texts = [d.page_content for d in sampled_chunks]

# ---- Sparse (TF-IDF; no NLTK) ----
from sklearn.feature_extraction.text import TfidfVectorizer
token_pattern = r"[A-Za-z0-9]+"
vectorizer = TfidfVectorizer(
    token_pattern=token_pattern,
    lowercase=True,
    stop_words="english",
    min_df=2
)
X = vectorizer.fit_transform(corpus_texts)  # CSR matrix
print(f"TF-IDF built: {X.shape[0]} docs, vocab ~{X.shape[1]}")

# ---- Pinecone index (create if missing) — MUST be dotproduct for hybrid ----
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
base_name = os.getenv("PINECONE_INDEX_NAME", "ubc-grades")
index_name = f"{base_name}-hybrid"   # new index so we don't clash with old cosine index
dim = len(emb.embed_query("dimension check"))  # auto-detect

if index_name not in [x["name"] for x in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=dim,
        metric="dotproduct",  # <-- REQUIRED for dense+sparse hybrid
        spec=ServerlessSpec(
            cloud=os.getenv("PINECONE_CLOUD", "aws"),
            region=os.getenv("PINECONE_REGION", "us-east-1"),
        ),
    )
index = pc.Index(index_name)

# ---- Dense embeddings (batched + retries) ----
dense_vecs = adaptive_embed_documents(emb, corpus_texts)

# ---- Upsert hybrid (dense + sparse) ----
metas = [d.metadata | {"text": d.page_content} for d in sampled_chunks]

buf = []
for j, i_doc in enumerate(indices):
    row = X.getrow(j)
    idxs = row.indices.tolist()
    vals = row.data.astype("float32").tolist()

    buf.append({
        "id": f"{NAMESPACE}-{i_doc}",                 # keep original global index in the id
        "values": dense_vecs[j],                      # dense Gemini
        "sparse_values": {"indices": idxs, "values": vals},  # sparse TF-IDF
        "metadata": metas[j],
    })
    if len(buf) == UPSERT_BATCH:
        index.upsert(vectors=buf, namespace=NAMESPACE)
        buf = []
if buf:
    index.upsert(vectors=buf, namespace=NAMESPACE)

print(f"✅ Upserted {len(indices)} hybrid vectors to '{index_name}' (ns='{NAMESPACE}').")

# --- Hybrid query (dense + sparse with alpha) ---
def hybrid_query(index, vectorizer, emb, query: str, top_k=10, alpha=0.6, flt=None, namespace="ubc-grades"):
    # dense
    q_dense = l2_normalize(emb.embed_query(query))
    q_dense = [alpha * x for x in q_dense]
    # sparse
    q_row = vectorizer.transform([query])
    q_idx = q_row.indices.tolist()
    q_val = [(1 - alpha) * float(v) for v in q_row.data.tolist()]
    # search
    return index.query(
        vector=q_dense,
        sparse_vector={"indices": q_idx, "values": q_val},
        top_k=top_k,
        include_metadata=True,
        filter=flt,
        namespace=namespace
    )

# quick smoke test:
res = hybrid_query(index, vectorizer, emb, "cpsc 221 2023w average", top_k=5, alpha=0.6)
[m["metadata"].get("text","")[:120] for m in res.get("matches",[])]


Indexing first 48249/482499 chunks (fraction=0.10)


I0000 00:00:1756585633.496404   81354 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


TF-IDF built: 48249 docs, vocab ~13374
Embedded 8000/48249 (batch=64)
Embedded 16000/48249 (batch=64)
Embedded 24000/48249 (batch=64)
Embedded 32000/48249 (batch=64)
Embedded 40000/48249 (batch=64)
Embedded 48000/48249 (batch=64)
Embedded 48249/48249 (batch=64)
✅ Upserted 48249 hybrid vectors to 'ubc-grades-hybrid' (ns='ubc-grades').


['Campus: UBCV\nYear: 2023\nSession: W\nSubject: CPSC\nCourse: 221\nDetail: \nSection: 101\nTitle: Basic Algorithms and Data Stru',
 'Campus: UBCV\nYear: 2023\nSession: W\nSubject: CPSC\nCourse: 221\nDetail: \nSection: 203\nTitle: Basic Algorithms and Data Stru',
 'Campus: UBCV\nYear: 2024\nSession: W\nSubject: CPSC\nCourse: 221\nDetail: \nSection: 201\nTitle: Basic Algorithms and Data Stru',
 'Campus: UBCV\nYear: 2024\nSession: W\nSubject: CPSC\nCourse: 221\nDetail: \nSection: 101\nTitle: Basic Algorithms and Data Stru',
 'Campus: UBCV\nYear: 2023\nSession: W\nSubject: CPSC\nCourse: 221\nDetail: \nSection: 201\nTitle: Basic Algorithms and Data Stru']

In [9]:
from langchain_core.documents import Document

class PineconeHybridRetriever:
    def __init__(self, index, vectorizer, emb, namespace="ubc-grades", alpha=0.6, top_k=8, flt=None):
        self.index, self.vectorizer, self.emb = index, vectorizer, emb
        self.namespace, self.alpha, self.top_k, self.flt = namespace, alpha, top_k, flt

    def invoke(self, query: str):
        # dense part
        import math
        def l2(v): 
            n = math.sqrt(sum(x*x for x in v)) or 1.0
            return [x/n for x in v]
        q_dense = [self.alpha * x for x in l2(self.emb.embed_query(query))]
        # sparse part
        q_row = self.vectorizer.transform([query])
        q_idx = q_row.indices.tolist()
        q_val = [(1 - self.alpha) * float(v) for v in q_row.data.tolist()]
        res = self.index.query(
            vector=q_dense,
            sparse_vector={"indices": q_idx, "values": q_val},
            top_k=self.top_k,
            include_metadata=True,
            filter=self.flt,
            namespace=self.namespace,
        )
        docs = []
        for m in res.get("matches", []):
            md = m.get("metadata", {}) or {}
            txt = md.pop("text", "")
            docs.append(Document(page_content=txt, metadata=md | {"_score": m.get("score")}))
        return docs

retriever = PineconeHybridRetriever(index, vectorizer, emb, alpha=0.6, top_k=8)


In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-flash",  # or pro
    google_api_key=os.getenv("GEMINI_API_KEY"),
    temperature=0.2,
)

PROMPT = ChatPromptTemplate.from_messages([
    ("system",
     "You are a helpful assistant for UBC grades. Use the provided context. "
     "If the course/term/instructor is missing, ask for it. If unknown, say so."),
    ("human", "Question: {question}\n\nContext:\n{context}\n\nAnswer with citations (course/term).")
])

def format_docs(docs):
    out = []
    for d in docs[:10]:
        c = d.metadata.get("course", "")
        t = d.metadata.get("term", d.metadata.get("year", ""))
        out.append(f"- [{c} {t}] {d.page_content[:550]}")
    return "\n".join(out)

def answer(question: str):
    ctx_docs = retriever.invoke(question)
    return llm.invoke(PROMPT.format_messages(question=question, context=format_docs(ctx_docs))).content

print(answer("What was the average for CPSC 221 in 2023W?"))


I0000 00:00:1756590913.570250   81354 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1756590913.592141   81354 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


The average for CPSC 221 in 2023W varied depending on the section:

* **Section 101:** 80.9 (CPSC 221, 2023W)
* **Section 201:** 79.0 (CPSC 221, 2023W)
* **Section 202:** 78.5 (CPSC 221, 2023W)
* **Section 203:** 81.1 (CPSC 221, 2023W)

There is no single average for the entire course in 2023W.

